In [1]:
import pandas as pd
import numpy as np
import pickle

# **Load Model**

In [2]:
filename = 'model_gbc.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [3]:
loaded_model

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('nominal_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot_encoding',
                                                                   OneHotEncoder(dtype=<class 'int'>))]),
                                                  ['contract', 'dependents',
                                                   'onlinesecurity',
                                                   'onlinebackup',
                                                   'internetservice',
                                                   'deviceprotection',
                                                   'techsupport',
                                                   'paperlessbilling']),
                                                 ('numerical_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['tenure',
                                                   'monthlycharges'])])),
                ('balancing', RandomOverSampler(random_state=2024)),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.001, max_depth=1,
                                            max_features=6, n_estimators=50,
                                            random_state=2024))])

# **Dummy Observation**

In [46]:
new_cust = pd.DataFrame([
    {
        'dependents': 'No',
        'tenure': 23,
        'onlinesecurity': 'No',
        'onlinebackup': 'No',
        'internetservice': 'DSL',
        'deviceprotection': 'No',
        'techsupport': 'Yes',
        'contract': 'One year',
        'paperlessbilling': 'No',
        'monthlycharges': 111.90
    },
    {
        'dependents': 'Yes',
        'tenure': 9,
        'onlinesecurity': 'No',
        'onlinebackup': 'No',
        'internetservice': 'DSL',
        'deviceprotection': 'No',
        'techsupport': 'No',
        'contract': 'Month-to-month',
        'paperlessbilling': 'Yes',
        'monthlycharges': 20.15
    }
])

new_cust

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges
0,No,23,No,No,DSL,No,Yes,One year,No,111.90
1,Yes,9,No,No,DSL,No,No,Month-to-month,Yes,20.15


In [42]:
# Making predictions on new_cust
predictions = loaded_model.predict(new_cust)

# Converting predictions to human-readable format ("Yes" for churn, "No" for no churn)
predictions_readable = ['Yes' if pred == 1 else 'No' for pred in predictions]

# Adding predictions to new_cust DataFrame
new_cust['Churn'] = predictions_readable

In [43]:
new_cust

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges,Churn
0,No,23,No,No,DSL,No,Yes,One year,No,111.90,No
1,Yes,9,No,No,DSL,No,No,Month-to-month,Yes,20.15,Yes


# **Predict on the Test Dataset**

In [51]:
x_test_preprocessed = pd.read_csv('x_test_preprocessed.csv')

In [54]:
def predict_churn(new_data, loaded_model):
    predictions = loaded_model.predict(new_data)
    predictions_readable = ['Yes' if pred == 1 else 'No' for pred in predictions]
    new_data['Churn'] = predictions_readable
    return new_data

In [56]:
new_cust_predicted = predict_churn(x_test_preprocessed, loaded_model)
new_cust_predicted

,dependents,tenure,onlinesecurity,onlinebackup,internetservice,deviceprotection,techsupport,contract,paperlessbilling,monthlycharges,Churn
0,No,60,No,Yes,Fiber optic,Yes,No,Two year,Yes,99.65,No
1,No,9,Yes,No,Fiber optic,No,No,Month-to-month,Yes,74.25,Yes
2,No,68,Yes,Yes,DSL,Yes,Yes,Two year,Yes,90.80,No
3,Yes,19,No,No,Fiber optic,No,Yes,One year,Yes,100.95,No
4,No,1,No internet service,No internet service,No,No internet service,No internet service,Month-to-month,No,20.65,No
...,...,...,...,...,...,...,...,...,...,...,...
966,No,24,No,No,Fiber optic,No,No,Month-to-month,No,70.00,Yes
967,Yes,44,No,Yes,DSL,No,No,One year,Yes,77.55,No
968,No,32,Yes,No,DSL,No,No,One year,No,74.75,No
969,No,24,No,Yes,Fiber optic,Yes,No,Month-to-month,Yes,104.15,Yes
